# Oil & Commodity Price Analysis with OpenBB and OilPriceAPI

This notebook demonstrates how to fetch, analyze, and visualize oil and commodity prices using the OpenBB Platform with the OilPriceAPI provider extension.

**What you'll learn:**
- Fetch real-time commodity prices
- Analyze price spreads between crude oil benchmarks
- Visualize historical price trends
- Compare energy commodities

## Setup

First, install the required packages:

In [ ]:
!pip install openbb openbb-oilpriceapi -q

In [ ]:
from openbb import obb
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Configure your API key (get free at https://oilpriceapi.com/signup)
obb.user.credentials.oilpriceapi_api_key = "YOUR_API_KEY_HERE"

## 1. Fetch Current Prices

Get real-time prices for all supported commodities:

In [ ]:
# Fetch all commodity prices
prices = obb.commodity.oil.price(provider="oilpriceapi")
df = prices.to_dataframe()
df

## 2. Crude Oil Price Comparison

Compare prices across different crude oil benchmarks:

In [ ]:
# Filter crude oils
crude_symbols = ['WTI', 'BRENT', 'URALS', 'DUBAI']
crude_df = df[df['symbol'].isin(crude_symbols)].copy()

# Create bar chart
fig, ax = plt.subplots(figsize=(10, 6))
colors = ['#2ecc71' if x >= 0 else '#e74c3c' for x in crude_df['change_percent']]
bars = ax.bar(crude_df['symbol'], crude_df['price'], color=colors, edgecolor='black')

# Add price labels
for bar, price, change in zip(bars, crude_df['price'], crude_df['change_percent']):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
            f'${price:.2f}\n({change:+.2f}%)', ha='center', va='bottom', fontsize=10)

ax.set_ylabel('Price (USD/barrel)', fontsize=12)
ax.set_title('Crude Oil Benchmark Prices', fontsize=14)
ax.set_ylim(0, max(crude_df['price']) * 1.15)
plt.tight_layout()
plt.show()

## 3. Calculate Price Spreads

Spreads between benchmarks are important indicators for traders:

In [ ]:
# Get prices as dict for easy access
price_dict = dict(zip(crude_df['symbol'], crude_df['price']))

# Calculate spreads
spreads = {
    'Brent-WTI': price_dict.get('BRENT', 0) - price_dict.get('WTI', 0),
    'Brent-Urals': price_dict.get('BRENT', 0) - price_dict.get('URALS', 0),
    'Dubai-WTI': price_dict.get('DUBAI', 0) - price_dict.get('WTI', 0),
}

print("=== Crude Oil Spreads ===")
for spread_name, value in spreads.items():
    print(f"{spread_name}: ${value:+.2f}/barrel")

## 4. Historical Price Analysis

Analyze WTI price trends over the past month:

In [ ]:
# Fetch historical data
history = obb.commodity.oil.historical(
    symbol="WTI",
    period="past_month",
    provider="oilpriceapi"
)
hist_df = history.to_dataframe()
hist_df.head()

In [ ]:
# Plot historical prices
fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(hist_df['date'], hist_df['price'], marker='o', linewidth=2, markersize=4, color='#3498db')
ax.fill_between(hist_df['date'], hist_df['price'], alpha=0.3, color='#3498db')

# Add moving average
if len(hist_df) >= 7:
    hist_df['ma7'] = hist_df['price'].rolling(window=7).mean()
    ax.plot(hist_df['date'], hist_df['ma7'], linestyle='--', color='#e74c3c', label='7-day MA')
    ax.legend()

ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Price (USD/barrel)', fontsize=12)
ax.set_title('WTI Crude Oil - Past Month', fontsize=14)
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 5. Energy Sector Overview

Compare all energy commodities:

In [ ]:
# Group by category
categories = {
    'Crude Oil': ['WTI', 'BRENT', 'URALS', 'DUBAI'],
    'Natural Gas': ['NG', 'NG_EU', 'NG_UK'],
    'Other': ['COAL', 'DIESEL_US', 'GASOLINE_US']
}

# Calculate category averages
print("=== Daily Change by Category ===")
for cat_name, symbols in categories.items():
    cat_df = df[df['symbol'].isin(symbols)]
    if not cat_df.empty:
        avg_change = cat_df['change_percent'].mean()
        print(f"{cat_name}: {avg_change:+.2f}%")

In [ ]:
# Visualize daily changes
fig, ax = plt.subplots(figsize=(12, 6))

colors = ['#2ecc71' if x >= 0 else '#e74c3c' for x in df['change_percent']]
bars = ax.barh(df['symbol'], df['change_percent'], color=colors, edgecolor='black')

ax.axvline(x=0, color='black', linewidth=0.5)
ax.set_xlabel('Daily Change (%)', fontsize=12)
ax.set_title('Commodity Price Changes Today', fontsize=14)
ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

## Summary Statistics

In [ ]:
# Historical statistics
print("=== WTI Historical Statistics (Past Month) ===")
print(f"Mean:   ${hist_df['price'].mean():.2f}")
print(f"Std:    ${hist_df['price'].std():.2f}")
print(f"Min:    ${hist_df['price'].min():.2f}")
print(f"Max:    ${hist_df['price'].max():.2f}")
print(f"Range:  ${hist_df['price'].max() - hist_df['price'].min():.2f}")

## Resources

- **OilPriceAPI**: [oilpriceapi.com](https://oilpriceapi.com) - Get your free API key
- **OpenBB Provider**: [PyPI](https://pypi.org/project/openbb-oilpriceapi/) | [GitHub](https://github.com/OilpriceAPI/openbb-oilpriceapi)
- **OpenBB Platform**: [openbb.co](https://openbb.co)